In [1]:
import os
import re
import numpy as np
import wfdb
import pickle
import scipy
from torch.utils.data import Dataset
import torch

def extract_segment_with_padding(z, k, N):
    start_idx = k - N
    end_idx = k + N + 1
    if start_idx < 0:
        padding_left = np.median(z[:end_idx])
        segment = np.concatenate([np.full(-start_idx, padding_left), z[:end_idx]])
    elif end_idx > len(z):
        padding_right = np.median(z[start_idx:])
        segment = np.concatenate([z[start_idx:], np.full(end_idx - len(z), padding_right)])
    else:
        segment = z[start_idx:end_idx]
    return segment

class MIT_BIH_Arythmia(Dataset):
    def __init__(self, N, M, dataset_dir='Datasets/files/', fs=10, output_dir="processed_data/", histogram_path=None):
        self.N = N
        self.M = M
        if histogram_path and os.path.exists(histogram_path):
            with open(histogram_path, 'rb') as f:
                self.cumulative_histogram = pickle.load(f)
            print("Załadowano histogram:", histogram_path)
        else:
            os.makedirs(output_dir, exist_ok=True)
            exclusion_lst = [str(i) for i in range(10, 11)]
            self.cumulative_histogram = []
            start_idx = 0
            for file in os.listdir(dataset_dir):
                name = re.match(r'^(.*\d\d+)\.atr$', file)
                if name and name.group(1) not in exclusion_lst:
                    print(f"Przetwarzanie: {name.group(1)}")
                    record = wfdb.rdsamp(f"{dataset_dir}{name.group(1)}")
                    annotation = wfdb.rdann(f"{dataset_dir}{name.group(1)}", 'atr')
                    signal = record[0][:, 0]
                    fs_original = record[1]["fs"]
                    num_samples_target = int(signal.shape[0] * fs / fs_original)
                    resampled_signal = scipy.signal.resample(signal, num_samples_target)
                    annotation_times_resampled = (annotation.sample * fs) / fs_original
                    data = {
                        "rec": resampled_signal,
                        "ann": {
                            "sample": annotation_times_resampled.astype(int).tolist(),
                            "aux_note": annotation.aux_note
                        }
                    }
                    output_filename = os.path.join(output_dir, f"{name.group(1)}.pkl")
                    with open(output_filename, 'wb') as f:
                        pickle.dump(data, f)
                    num_samples = len(data["ann"]["sample"])
                    self.cumulative_histogram.append((start_idx, start_idx + num_samples, output_filename))
                    start_idx += num_samples
            histogram_path = os.path.join(output_dir, "cumulative_histogram.pkl")
            with open(histogram_path, 'wb') as f:
                pickle.dump(self.cumulative_histogram, f)
            print("Przetwarzanie zakończone. Dane zapisane w:", output_dir)

    def __len__(self):
        return self.cumulative_histogram[-1][1]

    def __getitem__(self, idx):
    # Przejdź przez histogram skumulowany, aby znaleźć odpowiedni plik i zakres indeksów
        for start, end, filename in self.cumulative_histogram:
            if start <= idx < end:
                local_idx = idx - start  # Oblicz indeks lokalny w danym pliku
                break
        else:
            raise IndexError("Index out of range")  # Jeśli nie znajdziesz odpowiedniego zakresu, zgłoś błąd
        
        # Załaduj dane z odpowiedniego pliku
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        
        # Pobierz sygnał EKG i informacje o annotacjach
        rec = data["rec"]
        sample_idx = data["ann"]["sample"][local_idx]  # Indeks próbki w danym pliku
        aux_note = data["ann"]["aux_note"][local_idx]  # Etykieta (np. AFIB lub NORMAL)
        
        # Wyciąć odpowiedni segment EKG wokół punktu annotacji
        segment = extract_segment_with_padding(rec, sample_idx, self.N)
        
        # Ustal etykietę: 1 dla AFIB, 0 dla NORMAL
        label = 1 if aux_note == '(AFIB' else 0
        return torch.Tensor(segment).unsqueeze(0), label

    def count_afibs(self):
        afib_count = 0
        for start, end, filename in self.cumulative_histogram:
            # Załaduj dane z pliku
            with open(filename, 'rb') as f:
                data = pickle.load(f)
            
            # Sprawdź wszystkie etykiety i policz AFIB
            for aux_note in data["ann"]["aux_note"]:
                if "(AFIB" in aux_note:
                    afib_count += 1
        return afib_count


In [2]:
ds = MIT_BIH_Arythmia(100,5,fs=100,dataset_dir='Datasets/temp/physionet.org/files/ltafdb/1.0.0/')

Przetwarzanie: 00
Przetwarzanie: 01
Przetwarzanie: 03
Przetwarzanie: 05
Przetwarzanie: 06
Przetwarzanie: 07
Przetwarzanie: 08
Przetwarzanie: 100
Przetwarzanie: 101
Przetwarzanie: 102
Przetwarzanie: 103
Przetwarzanie: 104
Przetwarzanie: 105
Przetwarzanie: 11
Przetwarzanie: 110
Przetwarzanie: 111
Przetwarzanie: 112
Przetwarzanie: 113
Przetwarzanie: 114
Przetwarzanie: 115
Przetwarzanie: 116
Przetwarzanie: 117
Przetwarzanie: 118
Przetwarzanie: 119
Przetwarzanie: 12
Przetwarzanie: 120
Przetwarzanie: 121
Przetwarzanie: 122
Przetwarzanie: 13
Przetwarzanie: 15
Przetwarzanie: 16
Przetwarzanie: 17
Przetwarzanie: 18
Przetwarzanie: 19
Przetwarzanie: 20
Przetwarzanie: 200
Przetwarzanie: 201
Przetwarzanie: 202
Przetwarzanie: 203
Przetwarzanie: 204
Przetwarzanie: 205
Przetwarzanie: 206
Przetwarzanie: 207
Przetwarzanie: 208
Przetwarzanie: 21
Przetwarzanie: 22
Przetwarzanie: 23
Przetwarzanie: 24
Przetwarzanie: 25
Przetwarzanie: 26
Przetwarzanie: 28
Przetwarzanie: 30
Przetwarzanie: 32
Przetwarzanie: 33


## Simple model

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image
import re
import wfdb
import wfdb.processing
import scipy
from torch.utils.data import Dataset
import numpy as np
import json
import torch.nn as nn
import torch
from tqdm import tqdm
import torch.nn.functional as F

In [ ]:
class SimpleConv(nn.Module):
    def __init__(self, input = 201, input_ch = 1, num_classes = 2):
        super(SimpleConv, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(input_ch, 64, kernel_size=7, padding='same'),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=3, padding='same'),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Conv1d(64, 128, kernel_size=3, padding='same'),      # out 1 x 128 x n
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2),                        # out 1 x 128 x n//2
            nn.Conv1d(128, 128, kernel_size=3, padding='same'),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Conv1d(128, 256, kernel_size=3, padding='same'),     # out 1 x 256 x n//2
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2),                        # out 1 x 256 x n//4
            nn.Conv1d(256, 256, kernel_size=3, padding='same'),     
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Conv1d(256, 512, kernel_size=3, padding='same'),     
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(2),                        # out 1 x 512 x n//8
            nn.Flatten(),
            nn.Linear(512*(input//8), 256),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes),
        )
        self.model.to('cuda:0')

    def forward(self, x):

        return self.model(x)
    
    def train_model(self, train_loader, valid_loader, num_epochs = 5, learning_rate=0.001, save_best = False, save_thr = 0.94):
        best_accuracy = 0.0
        total_step = len(train_loader)
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.RMSprop(self.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

        for epoch in range(num_epochs):
            # self.train()
            correct = 0
            total = 0
            for i, (images, labels) in enumerate(tqdm(train_loader)):
                # Move tensors to the configured device
                images = images.float().to("cuda")
                labels = labels.type(torch.LongTensor)
                labels = labels.to("cuda")


                optimizer.zero_grad()

                # Forward pass
                outputs = self.forward(images)
                loss = criterion(outputs, labels)
                # Backward and optimize
                loss.backward()
                
                optimizer.step()

                # accuracy
                _, predicted = torch.max(outputs.data, 1)
                correct += (torch.eq(predicted, labels)).sum().item()
                total += labels.size(0)

                del images, labels, outputs

            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.4f}'
                            .format(epoch+1, num_epochs, i+1, total_step, loss.item(), (float(correct))/total))


            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            # Validation
            with torch.no_grad():
                correct = 0
                total = 0
                for images, labels in valid_loader:
                    images = images.float().to("cuda")
                    labels = labels.to("cuda")
                    outputs = self.forward(images)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (torch.eq(predicted, labels)).sum().item()
                    del images, labels, outputs
                if(((100 * correct / total) > best_accuracy) and save_best and ((100 * correct / total) > save_thr)):
                    torch.save(self.state_dict(), "best_resnet50_MINST-DVS2.pt")

                print('Accuracy of the network: {} %'.format( 100 * correct / total))

In [5]:
model = SimpleConv()

In [ ]:
from torch.utils.data import DataLoader, random_split
train_set, val_set = random_split(ds, [0.8, 0.2])
train = DataLoader(train_set, batch_size=32, shuffle=True, num_workers=4)
val = DataLoader(val_set, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
model.train_model(train,val,num_epochs=90)

  0%|          | 0/223721 [00:00<?, ?it/s]